In [ ]:
import pandas as pd
from utils import *
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split, KFold, cross_val_score, LeaveOneOut,GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Vegetation Classification using Machine Learning

## Load Data

In [ ]:
# Load all the classes into different Dataframes

class1 = pd.read_csv('data/Data_Class_1.csv')
class3 = pd.read_csv('data/Data_Class_3.csv')
class6 = pd.read_csv('data/Data_Class_6.csv')

## Descriptive Statistics

In [ ]:
# Combine all classes into one dataset and view the top rows

data = pd.concat([class1, class3, class6], ignore_index=True)
data

In [ ]:
# Check data types and non-null counts

data.info()

According to this information:
- 5940 rows
- 18 columns
- No null values
- 3 categorical columns and 15 numerical columns

### Numerical Columns
View some metrics of each numerical column

In [ ]:
# Generate descriptive statistics of the numerical columns

data.describe()

As displayed above, we can identify the following data characteristics:
- The columns have different ranges of values
- Some columns like the horizontal_distance_to_water have a significant difference between the mean and the median, indicating a possible skewness in the data
- When comparing the third quartile and the maximum value, we can see that some columns have outliers since the maximum value is significantly higher than the third quartile
- The same can be said for the minimum value and the first quartile, where the minimum value is significantly lower than the first quartile
- The standard deviation of the features possess a wide range of magnitudes, indicating that in some columns the data is spread out from the mean

### Categorical columns

In [ ]:
# Generate descriptive statistics of the categorical columns

data.describe(include='object')

The categorical columns have the following characteristics:
- We have two columns with few unique values and one column with many unique values. We need to take this into account when encoding the data.
- The most frequent values of each column are present in at least 25% of the rows

## Univariate Analysis

In [ ]:
# Get the name of the numerical and categorical columns

numerical_columns = data.select_dtypes(exclude='object').columns.drop('Id')
categorical_columns = data.select_dtypes(include='object').columns

In [ ]:
# Draw the boxplot visualizations of the numerical columns

boxplot_visualization(data, numerical_columns, 'Boxplot visualization of the numerical columns')

- Several columns such as the Horizontal_Distance_To_Water, Horizontal_Distance_To_Roadways, and Horizontal_Distance_To_Fire_Points have a significant number of outliers. 
These outliers may appear since different types of vegetation may have very distinct characteristics and values linked to them. If further investigation reveals that it is not the case, they may affect the performance of the models and it might be important to consider removing them.
- Given the length of the boxes, the majority of the data contains less variability since have narrow boxes. The Slope_Orientation column has a wide box, so we can identify that the values have more variability.

In [ ]:
# Draw the barplot visualizations of the categorical columns

barplot_visualization(data, categorical_columns, 'Barplot visualization of the categorical columns')

- Given the distribution of the categorical columns there are clear outliers in the Soil_Type and Wilderness_Area.
- Our classes appear in similar frequencies and it suggests that we will not encounter issues related to class imbalance, which is a crucial information to have in mind when evaluating the models.

## Bivariate Analysis

In [ ]:
# Encode the categorical columns

encoded_data = data.copy()
encoded_data['Soil_Type_Enc'] = pd.factorize(data['Soil_Type'])[0]
encoded_data['Wilderness_Area_Enc'] = pd.factorize(data['Wilderness_Area'])[0]
encoded_data['Vegetation_Type_Enc'] = pd.factorize(data['Vegetation_Type'])[0]

In [ ]:
# Generate descriptive statistics of data after encoding

encoded_data.describe()

The encoded data has the following characteristics:
- The columns have different ranges of values, since the encoding was done based on the number of unique values in each column
- The mean and median values are close to each other, indicating that the data is not skewed
- The column with the highest standard deviation is the Soil_Type_Enc column, indicating that the data is spread out from the mean


In [ ]:
# Remove Id for correlation matrix

encoded_data_without_id = encoded_data.drop('Id', axis=1)

In [ ]:
# Generate heatmap of the data

heatmap_visualization(encoded_data_without_id, 'Heatmap visualization')

The correlation of the features with the target column varies a lot and we can see that we have both highly correlated and uncorrelated features. The first ones may be the most important for our analysis, while the second ones may not provide any benefit.
In the next step, the columns with close to 0 correlation with the target column will be deleted, seeing as these do not provide any benefit to our analysis.

This applies to the following columns:
- Canopy_Density
- Rainfall_Summer
- Rainfall_Winter
- Wind_Exposure_Level

In [ ]:
# Removing the columns without any correlation

encoded_data_without_id = encoded_data_without_id.drop(['Canopy_Density', 'Rainfall_Summer', 'Rainfall_Winter', 'Wind_Exposure_Level'], axis=1)

In [ ]:
# Generate the heatmap of the data after removing the columns

heatmap_visualization(encoded_data_without_id, 'Heatmap visualization')

- Without the columns that do not correlate with the target column, we can see that the correlation between the features and the target column is more evident. All these columns may help, to some extent, to predict the target column.

In [ ]:
# Display the correlation matrix

corr_matrix = encoded_data_without_id.corr(numeric_only=True)
corr_matrix['Vegetation_Type_Enc'].sort_values()

In [ ]:
# Draw the boxplot visualizations of the numerical columns by Vegetation type (Target variable)

boxplot_by_type_visualization(data, numerical_columns, 'Boxplot visualization of the numerical columns by Vegetation type')

- In the boxplots there are some columns that have outliers for each different Vegetation_Type.
- Some features like the Altitude, Horizontal_Distance_To_Roadways, slope and Horizontal_Distance_To_Fire_Points have very distinct boxes for the Type_1 of the target variable. This indicates that these columns might be very good predictors of the label.

In [ ]:
# Draw the crosstable visualizations of the numerical columns by Vegetation type (Target variable)

crosstab_by_type_visualization(data, categorical_columns, 'Barplot visualization of the categorical columns by Vegetation type')

- The crosstables show that some values of the categorical features are more frequent or even exclusive in some classes than in others. This indicates that these columns may be good predictors of the target variable.

## Methods Application

In [ ]:
# Load the models to be evaluated

models = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis()
}

### Data Preparation

In [ ]:
# Divide the data into features and target variable and separate the training and test data

X = encoded_data.drop(columns=['Vegetation_Type', 'Soil_Type', 'Wilderness_Area','Vegetation_Type_Enc', 'Id'])
y = encoded_data['Vegetation_Type_Enc']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Rescales each feature to have a mean of 0 and a standard deviation of 1

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Dataframe to store the metrics for each model and method

data_results = pd.DataFrame(columns=['Model', 'Method', 'Accuracy', 'Precision', 'Recall', 'F1', 'Std Dev', 'Confusion Matrix'])

- Since our dataset is balanced, we can use the accuracy metric to evaluate the models since it gives us a good overview of the model's performance.
- It is also useful to use the precision, recall to ensure that the model is not biased towards a specific class. Given the fact that we want to balance the precision and recall, the F1 score is a good metric to use.
- The confusion matrix will help identify misclassifications since it shows the number of correct and incorrect predictions for each class.

### Holdout Method

In [ ]:
holdout_evaluation(data_results,models, X_train, X_test, y_train, y_test)

### Cross Validation (K=5)

In [ ]:
cross_validation_evaluation(data_results,models, X, y, 5)

### Cross Validation (K=10)

In [ ]:
cross_validation_evaluation(data_results,models, X, y, 10)

### Leave One Out Cross Validation (LOOCV)

In [ ]:
#loocv_evaluation(data_results, models, X, y)

### Bootstrap

In [ ]:
bootstrap_evaluation(data_results, models,X, y, 100)

### Results 

In [ ]:
# Display the dataframe with the metrics for each model and method

data_results

- As we can see above, the models have similar results since the accuracy, precision, recall and F1 scores are in the range of 0.78 to 0.80. This can indicate that the performance of the models is consistent.
- As the variance equals to the square of the standard deviation, we can conclude that the resampling method have an impact on the results. The methods with the lowest variance are the Cross Validation with K=5 and the Bootstrap. Cross Validation with K=10 has the highest variance and Holdout method does not have a variance since it is a single split. 

In [ ]:
confusion_matrix_visualization(data_results, 'Confusion Matrix')

With theses confusion matrices, we can see that the all the models can easily identify the first Vegetation Type, while for the other two, it has more difficulty in distinguising between them, resulting in a lot of wrong predictions. This is expected considering that, as we saw in our analysis before, there aren't many features which provide a distinction between the Vegetation Types 3 and 6.

In order to improve our results, we need to study which features have a higher impact in the metrics for the models, and select the ones that provide the best results.

## Feature Selection

### Find the best range for C

In [ ]:
def best_feature_grid_search(X_train, y_train, parameters, model):
    grid = GridSearchCV(model, parameters, cv=KFold(n_splits=5, shuffle=True, random_state=42))

    grid.fit(scaler.transform(X_train), y_train)

    return grid.best_params_

In [ ]:
model = LogisticRegression()

#### Lasso

##### 0- 10

In [ ]:
parameters = {'C': np.arange(0, 10, 0.01), 'penalty': ['l1'], 'solver': ['saga']}
best_parameter_C = best_feature_grid_search(X_train, y_train, parameters, model)

In [ ]:
best_parameter_C

##### Rigde 

In [ ]:
#parameters = {'C': np.arange(0, 0.5, 0.01), 'penalty': ['l1']}

###### Lasso e Ridge

In [ ]:
parameters = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

best_feature_lasso_rigde = best_feature_grid_search(X_train, y_train, parameters, model)

###### Elastic Net

In [ ]:
parameters = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['elasticnet'], 'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'solver': ['saga']}
best_feature_elasticnet =best_feature_grid_search(X_train, y_train, parameters, model)

In [ ]:
best_models_lr ={
    'Logistic Regression with Lasso ou Ridge': LogisticRegression(C=best_feature_lasso_rigde['C'], penalty=best_feature_lasso_rigde['penalty']),
    'Logistic Regression with Elastic Net': LogisticRegression(C=best_feature_elasticnet['C'], penalty='elasticnet', l1_ratio=best_feature_elasticnet['l1_ratio'], solver='saga' )
}


### Holdout With Best Features

In [ ]:
holdout_evaluation(data_results,best_models_lr, X_train, X_test, y_train, y_test)

### Cross Validation (K=5) With Best Features

In [ ]:
cross_validation_evaluation(data_results, best_models_lr, X, y, 5)

### Cross Validation (K=10) With Best Features

In [ ]:
cross_validation_evaluation(data_results,best_models_lr, X, y, 10)

### Leave One Out Cross Validation (LOOCV) With Best Features

In [ ]:
#loocv_evaluation(data_results, best_models_lr, X, y, " with best features")

### Bootstrap With Best Features

In [ ]:
bootstrap_evaluation(data_results, best_models_lr, X, y, 100)

### Results

In [ ]:
data_results

In [ ]:
confusion_matrix_visualization(data_results, 'Confusion Matrix')

The results were not what we expected despite the small improvement since there is still a big misclassification betweeen the label 1 that is often classified as 2.

We tried getting the dummies of the categorical classes and only using the columns that had correlation with the objective above 0.3 since we considered that smaller values could indicate no correlation at all and few columns had values higher. Surprisingly the results did not improve.

Next we will apply the regularization methods and test the Ridge, Lasso and Elasticnet methods.

### Get  data with Dummies

In [ ]:
# Drop the encoded columns and get the dummies of the categorical columns

new_data =encoded_data_without_id.drop(
    columns=[
        'Vegetation_Type',
        'Soil_Type_Enc',
        'Wilderness_Area_Enc'
    ]
)
data_dummies = pd.get_dummies(new_data)

In [ ]:
# Generate the correlation matrix
corr_matrix = data_dummies.corr()
sorted_corr = corr_matrix['Vegetation_Type_Enc'].sort_values()

In [ ]:
# Plotting the sorted correlations
barplot_correlation_visualization(sorted_corr, 'Correlation with Vegetation_Type_Enc')

The correlation of the features with the target column varies a lot and we can see that we have both highly correlated and uncorrelated features. We will utilize the columns with higher correlation to predict the target column.

### Data Preparation with Dummies

In [ ]:
# Preparing the data to use in the regularization methods

X_dummies = data_dummies.drop(columns=['Vegetation_Type_Enc'])
y_dummies = data_dummies['Vegetation_Type_Enc']
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y_dummies, test_size=0.3, random_state=42)

### Data Preparation with Best Features and Dummies

In [ ]:
def best_feature_grid_search(X_train, y_train, parameters, model):
    grid = GridSearchCV(model, parameters, cv=KFold(n_splits=5, shuffle=True, random_state=42))

    grid.fit(X_train, y_train)

    return grid.best_params_

In [ ]:
model = LogisticRegression()

In [ ]:
parameters = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

best_feature_lasso_rigde = best_feature_grid_search(X_train, y_train, parameters, model)

In [ ]:
parameters = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['elasticnet'],
              'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'solver': ['saga']}

best_feature_elasticnet = best_feature_grid_search(X_train, y_train, parameters, model)

In [ ]:
best_models_lr = {
    'Logistic Regression with Lasso ou Ridge': LogisticRegression(C=best_feature_lasso_rigde['C'],
                                                                  penalty=best_feature_lasso_rigde['penalty']),
    'Logistic Regression with Elastic Net': LogisticRegression(C=best_feature_elasticnet['C'], penalty='elasticnet',
                                                               l1_ratio=best_feature_elasticnet['l1_ratio'],
                                                               solver='saga')
}

### Holdout Method

In [ ]:
holdout_evaluation(data_results, best_models_lr, X_train, X_test, y_train, y_test, " dummies")

### Cross Validation (K=5)

In [ ]:
cross_validation_evaluation(data_results, best_models_lr, X_dummies, y_dummies, 5, " dummies")

### Cross Validation (K=10)

In [ ]:
cross_validation_evaluation(data_results, best_models_lr, X_dummies, y_dummies, 10, " dummies")

### Leave One Out with Cross Validation

In [ ]:
#loocv_evaluation(data_results, best_models_lr, X_dummies, y_dummies, " dummies")

### Bootstrap

In [ ]:
bootstrap_evaluation(data_results, best_models_lr, X_dummies, y_dummies, 100, " dummies")

### Results with Dummies

In [ ]:
data_results

In [ ]:
confusion_matrix_visualization(data_results, 'Confusion Matrix')

The results were worse for the holdout method and improved for bootstrap with lasso / ridge. This means that we have our best model yet.